# Full demo of application insights

# Setup

Validate prerequisites...

In [23]:
import pandas,sys,os,json,html,getpass,time
pandas_version = pandas.__version__.split('.')
pandas_major = int(pandas_version[0])
pandas_minor = int(pandas_version[1])
pandas_patch = int(pandas_version[2])
if not (pandas_major > 0 or (pandas_major == 0 and pandas_minor > 24) or (pandas_major == 0 and pandas_minor == 24 and pandas_patch >= 2)):
    sys.exit('Please upgrade the Notebook dependency before you can proceed, you can do it by running the "Reinstall Notebook dependencies" command in command palette (View menu -> Command Palette…).')
def run_command(command):
    print("Executing: " + command)
    !{command}
    if _exit_code != 0:
        sys.exit(f'Command execution failed with exit code: {str(_exit_code)}.\n\t{command}\n')
    print(f'Successfully executed: {command}')

# Set variables

Change the variables acordingly.

In [24]:
# Azure
azure_subscription_id = 'b0aa5656-5655-449f-819a-684cd2453041'
azure_region = 'northeurope'
azure_resource_group = 'mslearn-appinsights'
# SQL Server
mssql_logical_name = 'mslearn-appinsights-sqlserver'
mssl_admin_user = 'administrador'
mssql_password = 'PaSSw0rdñ.' # Despite it´s actual value, the password is going to be asked by command prompt
mssql_auth_mode = 'basic'
mssql_database_name = 'AdventureWorksDemo'  # It will deploy the sample database AdventureWorksLT database schema
mssql_database_edition = 'Basic' # for more options https://docs.microsoft.com/en-us/cli/azure/sql/db?view=azure-cli-latest#az-sql-db-create 
# CosmosDB
cosmosdb_account_name = 'mslearn-appinsights-cosmosdb'
cosmosdb_database_name = 'database'
cosmosdb_container_name = 'cosmosdbcontainer'
cosmosdb_throughput = '400' #throughput for all containers in this setup

# Environment
os.environ["PATH"] = os.environ["PATH"] + ";" + "C:\\Program Files\\Docker\\Docker\\resources\\bin;C:\\Program Files (x86)\\Microsoft SDKs\\Azure\\CLI2\\wbin;C:\\Program Files (x86)\\Microsoft SDKs\\Azdata\\CLI\\wbin"
print('Variables have been set successfully.')


Variables have been set successfully.


## Password by command prompt

In [8]:
invoked_by_wizard = "PYTHON_VAR_ADMIN_PASSWORD" in os.environ
if invoked_by_wizard:
    mssql_password = os.environ["PYTHON_VAR_ADMIN_PASSWORD"]
else:
    mssql_password = getpass.getpass(prompt = 'Controller password')
    if mssql_password == "":
        sys.exit(f'Password is required.')
    confirm_password = getpass.getpass(prompt = 'Confirm password')
    if mssql_password != confirm_password:
        sys.exit(f'Passwords do not match.')
print('You can also use the controller password to access Knox and SQL Server.')

You can also use the controller password to access Knox and SQL Server.


# Login to Azure

In [25]:
run_command(f'az login')

Executing: az login
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "829b7506-5cd7-42a3-bd0f-29c3bfddf24c",
    "id": "4beccc95-4a09-4148-8055-6b5972ff400a",
    "isDefault": false,
    "managedByTenants": [],
    "name": "Visual Studio Ultimate con MSDN",
    "state": "Enabled",
    "tenantId": "829b7506-5cd7-42a3-bd0f-29c3bfddf24c",
    "user": {
      "name": "enrique-catala@outlook.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "829b7506-5cd7-42a3-bd0f-29c3bfddf24c",
    "id": "e1c3d108-7f5f-45f9-bbf7-7098514e425d",
    "isDefault": false,
    "managedByTenants": [],
    "name": "Plataformas de MSDN",
    "state": "Enabled",
    "tenantId": "829b7506-5cd7-42a3-bd0f-29c3bfddf24c",
    "user": {
      "name": "enrique-catala@outlook.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "829b7506-5cd7-42a3-bd0f-29c3bfddf24c",
    "id": "d4d355d3-0f93-4736-939a-4c75f3de6056",
    "isDefault": 

## Set active azure subscription

In [26]:
if azure_subscription_id != "":
    run_command(f'az account set --subscription {azure_subscription_id}')
else:
    print('Using the default Azure subscription', {azure_subscription_id})
run_command(f'az account show')

Executing: az account set --subscription b0aa5656-5655-449f-819a-684cd2453041
Successfully executed: az account set --subscription b0aa5656-5655-449f-819a-684cd2453041
Executing: az account show
{
  "environmentName": "AzureCloud",
  "homeTenantId": "829b7506-5cd7-42a3-bd0f-29c3bfddf24c",
  "id": "b0aa5656-5655-449f-819a-684cd2453041",
  "isDefault": true,
  "managedByTenants": [
    {
      "tenantId": "2f4a9838-26b7-47ee-be60-ccc1fdec5953"
    }
  ],
  "name": "Microsoft Azure Sponsorship",
  "state": "Enabled",
  "tenantId": "829b7506-5cd7-42a3-bd0f-29c3bfddf24c",
  "user": {
    "name": "enrique-catala@outlook.com",
    "type": "user"
  }
}
Successfully executed: az account show


# Create azure resource group

In [11]:
run_command(f'az group create --name {azure_resource_group} --location {azure_region}')

Executing: az group create --name mslearn-appinsights --location northeurope
{
  "id": "/subscriptions/b0aa5656-5655-449f-819a-684cd2453041/resourceGroups/mslearn-appinsights",
  "location": "northeurope",
  "managedBy": null,
  "name": "mslearn-appinsights",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}
Successfully executed: az group create --name mslearn-appinsights --location northeurope


# Create logical SQL Server

If you don´t have a valid SQL Server for the demo purposes, create your instance

In [17]:
run_command(f'az sql server create --name {mssql_logical_name} --resource-group {azure_resource_group} --location {azure_region} --admin-user "{mssl_admin_user}" --admin-password "{mssql_password}"')

Executing: az sql server create --resource-group mslearn-appinsights --location northeurope --name mslearn-appinsights-sqlserver --admin-user "administrador" --admin-password "PaSSw0rdñ."
{
  "administratorLogin": "administrador",
  "administratorLoginPassword": null,
  "fullyQualifiedDomainName": "mslearn-appinsights-sqlserver.database.windows.net",
  "id": "/subscriptions/b0aa5656-5655-449f-819a-684cd2453041/resourceGroups/mslearn-appinsights/providers/Microsoft.Sql/servers/mslearn-appinsights-sqlserver",
  "identity": null,
  "kind": "v12.0",
  "location": "northeurope",
  "name": "mslearn-appinsights-sqlserver",
  "resourceGroup": "mslearn-appinsights",
  "state": "Ready",
  "tags": null,
  "type": "Microsoft.Sql/servers",
  "version": "12.0"
}
Successfully executed: az sql server create --resource-group mslearn-appinsights --location northeurope --name mslearn-appinsights-sqlserver --admin-user "administrador" --admin-password "PaSSw0rdñ."


## Allow access to all Azure Services

In [25]:
run_command(f'az sql server firewall-rule create --name AllowAllAzureServices --start-ip-address 0.0.0.0 --end-ip-address 0.0.0.0 --server {mssql_logical_name} --resource-group {azure_resource_group}')
 

Executing: az sql server firewall-rule create --name AllowAllAzureServices --start-ip-address 0.0.0.0 --end-ip-address 0.0.0.0 --server mslearn-appinsights-sqlserver --resource-group mslearn-appinsights
{
  "endIpAddress": "0.0.0.0",
  "id": "/subscriptions/b0aa5656-5655-449f-819a-684cd2453041/resourceGroups/mslearn-appinsights/providers/Microsoft.Sql/servers/mslearn-appinsights-sqlserver/firewallRules/AllowAllAzureServices",
  "kind": "v12.0",
  "location": "North Europe",
  "name": "AllowAllAzureServices",
  "resourceGroup": "mslearn-appinsights",
  "startIpAddress": "0.0.0.0",
  "type": "Microsoft.Sql/servers/firewallRules"
}
Successfully executed: az sql server firewall-rule create --name AllowAllAzureServices --start-ip-address 0.0.0.0 --end-ip-address 0.0.0.0 --server mslearn-appinsights-sqlserver --resource-group mslearn-appinsights


# Deploy AdventureWorksLT sample database

The database restored will have the name you wrote in the "mssql_database_name" variable

In [23]:
run_command(f'az sql db create --name {mssql_database_name} --sample-name AdventureWorksLT --edition {mssql_database_edition} --server {mssql_logical_name} --resource-group {azure_resource_group} ')

Executing: az sql db create --name AdventureWorksDemo --sample-name AdventureWorksLT --edition Basic --server mslearn-appinsights-sqlserver --resource-group mslearn-appinsights 
{
  "autoPauseDelay": null,
  "catalogCollation": "SQL_Latin1_General_CP1_CI_AS",
  "collation": "SQL_Latin1_General_CP1_CI_AS",
  "createMode": null,
  "creationDate": "2020-03-06T12:11:39.910000+00:00",
  "currentServiceObjectiveName": "Basic",
  "currentSku": {
    "capacity": 5,
    "family": null,
    "name": "Basic",
    "size": null,
    "tier": "Basic"
  },
  "databaseId": "779802dc-8634-4224-aa4a-6dad18a35530",
  "defaultSecondaryLocation": "westeurope",
  "earliestRestoreDate": "2020-03-06T12:41:39.910000+00:00",
  "edition": "Basic",
  "elasticPoolId": null,
  "elasticPoolName": null,
  "failoverGroupId": null,
  "id": "/subscriptions/b0aa5656-5655-449f-819a-684cd2453041/resourceGroups/mslearn-appinsights/providers/Microsoft.Sql/servers/mslearn-appinsights-sqlserver/databases/AdventureWorksDemo",
  "

# Create CosmosDB account

This process will take 10m more or less

In [16]:
run_command(f'az cosmosdb create --name {cosmosdb_account_name} --resource-group {azure_resource_group} ')

Executing: az cosmosdb create --name mslearn-appinsights-cosmosdb --resource-group mslearn-appinsights 
{
  "capabilities": [],
  "connectorOffer": null,
  "consistencyPolicy": {
    "defaultConsistencyLevel": "Session",
    "maxIntervalInSeconds": 5,
    "maxStalenessPrefix": 100
  },
  "databaseAccountOfferType": "Standard",
  "disableKeyBasedMetadataWriteAccess": false,
  "documentEndpoint": "https://mslearn-appinsights-cosmosdb.documents.azure.com:443/",
  "enableAutomaticFailover": false,
  "enableCassandraConnector": null,
  "enableMultipleWriteLocations": false,
  "failoverPolicies": [
    {
      "failoverPriority": 0,
      "id": "mslearn-appinsights-cosmosdb-northeurope",
      "locationName": "North Europe"
    }
  ],
  "id": "/subscriptions/b0aa5656-5655-449f-819a-684cd2453041/resourceGroups/mslearn-appinsights/providers/Microsoft.DocumentDB/databaseAccounts/mslearn-appinsights-cosmosdb",
  "ipRangeFilter": "",
  "isVirtualNetworkFilterEnabled": false,
  "kind": "GlobalDocu

# Create CosmosDB database

In [17]:
run_command(f'az cosmosdb sql database create  --name {cosmosdb_database_name} --account-name {cosmosdb_account_name} --resource-group {azure_resource_group}  ')

Executing: az cosmosdb sql database create  --name database --account-name mslearn-appinsights-cosmosdb --resource-group mslearn-appinsights  
{
  "id": "/subscriptions/b0aa5656-5655-449f-819a-684cd2453041/resourceGroups/mslearn-appinsights/providers/Microsoft.DocumentDB/databaseAccounts/mslearn-appinsights-cosmosdb/sqlDatabases/database",
  "location": null,
  "name": "database",
  "resource": {
    "_self": "dbs/-MFUAA==/",
    "id": "database"
  },
  "resourceGroup": "mslearn-appinsights",
  "tags": null,
  "type": "Microsoft.DocumentDB/databaseAccounts/sqlDatabases"
}
Successfully executed: az cosmosdb sql database create  --name database --account-name mslearn-appinsights-cosmosdb --resource-group mslearn-appinsights  
Command group 'cosmosdb sql' is in preview. It may be changed/removed in a future release.


# Create cosmosdb container

In [27]:
run_command(f'az cosmosdb sql container create  --name {cosmosdb_container_name} --partition-key-path "/productid" --throughput {cosmosdb_throughput} --database-name {cosmosdb_database_name} --account-name {cosmosdb_account_name} --resource-group {azure_resource_group}  ')

Executing: az cosmosdb sql container create  --name cosmosdbcontainer --partition-key-path "/productid" --throughput 400 --database-name database --account-name mslearn-appinsights-cosmosdb --resource-group mslearn-appinsights  
{
  "id": "/subscriptions/b0aa5656-5655-449f-819a-684cd2453041/resourceGroups/mslearn-appinsights/providers/Microsoft.DocumentDB/databaseAccounts/mslearn-appinsights-cosmosdb/sqlDatabases/database/containers/cosmosdbcontainer",
  "location": null,
  "name": "cosmosdbcontainer",
  "resource": {
    "_conflicts": "conflicts/",
    "_docs": "docs/",
    "_self": "dbs/-MFUAA==/colls/-MFUAN67NjQ=/",
    "_sprocs": "sprocs/",
    "_triggers": "triggers/",
    "_udfs": "udfs/",
    "conflictResolutionPolicy": {
      "conflictResolutionPath": "/_ts",
      "conflictResolutionProcedure": "",
      "mode": "LastWriterWins"
    },
    "defaultTtl": null,
    "geospatialConfig": {
      "type": "Geography"
    },
    "id": "cosmosdbcontainer",
    "indexingPolicy": {
    

In [ ]:
print(f'Configuration end')